In [1]:
import random
import numpy as np
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer 
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Activation, Dropout # type: ignore
from tensorflow.keras.optimizers import SGD # type:ignore

In [2]:
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())


In [3]:
words= []
classes= []
documents= []
ignore_letters= ['?','!',',','.']

for intent in intents['intents']:
    for pattern in intent["patterns"]:
        word_list= nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes :
            classes.append(intent['tag'])


In [4]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
classes=sorted(set(classes))
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

In [5]:
training=[]
output_empty=[0]*len(classes)
for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
       bag.append(1) if word in word_patterns else bag.append(0)      
    output_row=list(output_empty)   
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])

In [6]:
random.shuffle(training)
training= np.array(training, dtype=object)

train_x=list(training[:,0])
train_y=list(training[:,1])

In [7]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd= SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
print('done')



c:\Users\GIGABYTE\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\GIGABYTE\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0556 - loss: 2.1781      
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0571 - loss: 2.1369     
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.2567 - loss: 2.0591
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3066 - loss: 1.9022 
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.3226 - loss: 1.8448
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6450 - loss: 1.6874 
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.5376 - loss: 1.6389
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - accuracy: 0.4804 - loss: 1.6350
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - accuracy: 0.7206 - loss: 1.3082
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.8086 - loss: 1.2349
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.8173 - loss: 1.1278
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accu

In [8]:
model.save('chatbot_model.keras')